In [1]:
# generate the monomials
varcounter=1; monom_str=[]; onevars=[]; Bohr={1:'1'}
for n in range(1,120):
    if is_prime(n):
        newvar='z'+str(varcounter); varcounter=varcounter+1
        monom_str=monom_str+[newvar]
        onevars=onevars+[newvar]
        Bohr[n]=newvar
    else:
        pfac=list(factor(n))
        newmonom='1'
        for (p,e) in pfac:
            newmonom = newmonom + '*' + Bohr[p] + '^'+ str(e)
        monom_str=monom_str+[newmonom]
#P=PolynomialRing(QQ,onevars)
#print(monom_str)
############
#
# establish extension field and evaluate monomials
#
###########
#S.<sqrt2,sqrt3,sqrt5,sqrt7,sqrt11,sqrt13,sqrt17> = QQ.extension([x^2-2, x^2-3, x^2-5, x^2-7, x^2-11, x^2-13, x^2-17])
#S.<A>=PolynomialRing(QQ)
#assume(A,'rational')
S=QQ
Q.<z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30>=PolynomialRing(S,order='negdeglex')
#sigh, I can't figure out how to make it appreciate the variables
############
varseval=[]
for n in Bohr.keys():
    varseval=varseval+[sage_eval(Bohr[n],locals=vars())]
varseval.remove(1)
print(varseval)
monomslist=[]
for monstr in monom_str:
    monomslist = monomslist + [sage_eval(monstr,locals=vars())]
print(monomslist)

[z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30]
[1, z1, z2, z1^2, z3, z1*z2, z4, z1^3, z2^2, z1*z3, z5, z1^2*z2, z6, z1*z4, z2*z3, z1^4, z7, z1*z2^2, z8, z1^2*z3, z2*z4, z1*z5, z9, z1^3*z2, z3^2, z1*z6, z2^3, z1^2*z4, z10, z1*z2*z3, z11, z1^5, z2*z5, z1*z7, z3*z4, z1^2*z2^2, z12, z1*z8, z2*z6, z1^3*z3, z13, z1*z2*z4, z14, z1^2*z5, z2^2*z3, z1*z9, z15, z1^4*z2, z4^2, z1*z3^2, z2*z7, z1^2*z6, z16, z1*z2^3, z3*z5, z1^3*z4, z2*z8, z1*z10, z17, z1^2*z2*z3, z18, z1*z11, z2^2*z4, z1^6, z3*z6, z1*z2*z5, z19, z1^2*z7, z2*z9, z1*z3*z4, z20, z1^3*z2^2, z21, z1*z12, z2*z3^2, z1^2*z8, z4*z5, z1*z2*z6, z22, z1^4*z3, z2^4, z1*z13, z23, z1^2*z2*z4, z3*z7, z1*z14, z2*z10, z1^3*z5, z24, z1*z2^2*z3, z4*z6, z1^2*z9, z2*z11, z1*z15, z3*z8, z1^5*z2, z25, z1*z4^2, z2^2*z5, z1^2*z3^2, z26, z1*z2*z7, z27, z1^3*z6, z2*z3*z4, z1*z16, z28, z1^2*z2^3, z29, z1*z3*z5, z2*z12, z1^4*z4, z30, z1*z2*z8, z3*z9, z1^2*z10, z2^2*z6

In [2]:
#the 'DICT' command I was previously using is outdated now, but I still need to be able to get the degrees
def multideg(monom): #take a monomial in z1 and z2 and return an ordered pair with its multidegree 
                    #(there's probably a way to do this more generally)
    z1deg=monom.degree(z1)
    z2deg=monom.degree(z2)
    return (z1deg,z2deg)
def coeffdict(poly,monomslist,varseval): #takes a polynomial (as an expression) and a pregenerated list of monomials and returns a dictionary
                                # keys are ordered tuples of degrees of a monomial, entries are coefficients
    polydict={(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): poly((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))}
    for monmindex in range(1,len(monomslist)):
        monomattempt=monomslist[monmindex]
        #print(monomattempt)
        multi=tuple(monomattempt.degree(thevar) for thevar in varseval)
        #print(multi)
        d={}
        for n in range(len(varseval)):
            d.update({varseval[n]:multi[n]})
        #print(d)
        polydict.update({multi:poly.coefficient(d)})
    return polydict

In [3]:
############
#
#functions to actually find the OPAs
#
############
def InnerProd(f,g,monomslist,varseval):
    cfsf=coeffdict(f,monomslist,varseval)#the coefficients paired into a dictionary with the multi-indices
    cfsg=coeffdict(g,monomslist,varseval)
    multi_indices = list(set(cfsf.keys()).union(set(cfsg.keys()))) #pretty sure unioning them is unnecessary, but *shrug*
    ip=0
    for alpha in multi_indices:
        if alpha in cfsf.keys():
            term = cfsf[alpha]
        else:
            term=0
        if alpha in cfsg.keys():
            term = term *(cfsg[alpha])
            #print(pairs[m],cfsg[pairs[m]])
        else:
            term=0
        ip=ip+term
    return ip
def buildGram(monoms,fcn,whichpn,varseval):
    M=matrix(Q,whichpn+1,lambda x,y: InnerProd(monoms[x]*fcn,monoms[y]*fcn,monoms,varseval))
    return M
def RHSmateq(f,whichpn):
    RHSveclist=[(f(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))]
    for n in range(whichpn):
        RHSveclist=RHSveclist+[0]
    #print(RHSveclist)
    return vector(RHSveclist)
def coeffpoly(optimal_list,whichOA,monomslist):
    S=QQ
    Q.<z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30>=PolynomialRing(S,order='negdeglex')
    thepoly='0'
    for coef in range(0,whichOA+1):
            thepoly=thepoly+'+'+str(optimal_list[coef])+'*'+str(monomslist[coef])
            #print((k,l),newcoefs[(k, l)])
    #return thepoly
    return sage_eval(thepoly,locals=locals())

In [4]:
b=(-4+3*z1-z1^2+3*z2-2*z1*z2+z1^2*z2-z2^2+z1*z2^2)
lazy1v=1-z1
show(lazy1v)
show(b)
Nikolski_k=1-z1    #These are the z(lambda-z^k) examples translated to DS as lambda-k^(-s), 
                   #which I've written here for k a prime and lambda=1
#Here are the z(lambda-z)^N examples translated to the polydisk with our DS monomial ordering and choosing lambda=L:
N2half=(1/2)^2 - 2*(1/2)*z1 + z2
N2L1=1-2*z1+z2
N2L2=2^2 - 2*2*z1 + z2
N2L3=3^2 - 2*3*z1 + z2
N2L4=4^2 - 2*4*z1 + z2
#
N3half=(1/2)^3 - 3*(1/2)^2*z1 + 3*(1/2)*z2 - z1^2
N3L1=1-3*z1+3*z2-z1^2
N3L2=2^3 - 3*2^2*z1 + 3*2*z2 - z1^2
N3L3=3^3 - 3*3^2*z1 + 3*3*z2 - z1^2
N3L4=4^4 - 3*4^2*z1 + 3*4*z2 - z1^2 
#L^4*z - 4*L^3*z^2 + 6*L^2*z^3 - 4*L*z^4 + z^5
N4L1=1-4*z1+6*z2-4*z1^2+z3    
#
#L^5*z - 5*L^4*z^2 + 10*L^3*z^3 - 10*L^2*z^4 + 5*L*z^5 - z^6
N5L1=1 - 5*z1 + 10*z2 - 10*z1^2 + 5*z3 - z1*z2
#
#L^10*z - 10*L^9*z^2 + 45*L^8*z^3 - 120*L^7*z^4 + 210*L^6*z^5 - 252*L^5*z^6 + 210*L^4*z^7 - 120*L^3*z^8 + 45*L^2*z^9 - 10*L*z^10 + z^11
N10L1=1-10*z1 + 45*z2 - 120*z1^2 + 210*z3 - 252*z1*z2 + 210*z4 - 120*z1^3 + 45*z2^2 - 10*z1*z3 + z5
# here are the ones with a + instead of a -
Np2=1+2*z1+z2
Np3=1+3*z1+3*z2+z1^2
Np4=1+4*z1+6*z2+4*z1^2+z3    
Np5=1 + 5*z1 + 10*z2 + 10*z1^2 + 5*z3 + z1*z2
Nikolski=[N2L1,N3L1,N4L1,N5L1,N10L1]
Nikolskiplus=[Np2,Np3,Np4,Np5]
NikolskiLs=[N2half,N2L1,N2L2,N2L3,N2L4,N3half,N3L1,N3L2,N3L3,N3L4]
for f in NikolskiLs:
    show(f)

1 - z1

-4 + 3*z1 + 3*z2 - z1^2 - 2*z1*z2 - z2^2 + z1^2*z2 + z1*z2^2

1/4 - z1 + z2

1 - 2*z1 + z2

4 - 4*z1 + z2

...

1/8 - 3/4*z1 + 3/2*z2 - z1^2

1 - 3*z1 + 3*z2 - z1^2

8 - 12*z1 + 6*z2 - z1^2

27 - 27*z1 + 9*z2 - z1^2

256 - 48*z1 + 12*z2 - z1^2

In [5]:
# going up to OA#16 gets us to n=15=z2*z3 so we at least see stuff beyond z1 and pure primes
f=lazy1v
#f=N3L4 #change which function in just one place
show(f)
#
#
fOAs=[]
fODs=[]
fOGs=[]
for whichOA in range(0,16):
    JAMmat=buildGram(monomslist,f,whichOA,varseval)
    other_side=RHSmateq(f,whichOA)
    JAM_aug = JAMmat.augment(other_side)
    JAM_red=JAM_aug.echelon_form()
    optimal_coeffs=JAM_red.column(whichOA+1)
    q=coeffpoly(optimal_coeffs,whichOA,monomslist)
    tonorm=q*f-1
    optimaldistsq=InnerProd(tonorm,tonorm,monomslist,varseval)
    #show(q)
    fOAs=fOAs+[q]
    fODs=fODs+[optimaldistsq]
for whichOA in range(1,len(fOAs)):
        fOGs=fOGs+[fOAs[whichOA]-fOAs[whichOA-1]]
#
#
#
#print "grammian"
#print ""
#print(latex(JAMmat))
show(fOGs)

1 - z1

KeyboardInterrupt: 

In [ ]:
print("test function")
print("\\begin{equation*}")
print("\\scalebox{2}{\\ensuremath{f=",latex(f),"}}")
print("\\end{equation*}")
deg=0
print("\\begin{align*}")
print('p_{',deg,'}&=',latex(fOAs[0]),'\\\\')
for deg in range(1,len(fOAs)):
    if fOAs[deg]!=fOAs[deg-1]:
        print('p_{',deg,'}&=',latex(fOAs[deg]),'\\\\')
        print('%%%%%%%%%%%%%%%%%%%')
    #deg=deg+1
print("\\end{align*}")
print("")
print("")
print("")

In [ ]:
alltheOGs=[]
for f in Nikolskiplus:
    #show(f)
    print("test function")
    print("\\begin{equation*}")
    print("\\scalebox{2}{\\ensuremath{f=",latex(f),"}}")
    print("\\end{equation*}")
    #
    #
    fOAs=[]
    fODs=[]
    for whichOA in range(0,16):
        JAMmat=buildGram(monomslist,f,whichOA,varseval)
        other_side=RHSmateq(f,whichOA)
        JAM_aug = JAMmat.augment(other_side)
        JAM_red=JAM_aug.echelon_form()
        optimal_coeffs=JAM_red.column(whichOA+1)
        q=coeffpoly(optimal_coeffs,whichOA,monomslist)
        tonorm=q*f-1
        optimaldistsq=InnerProd(tonorm,tonorm,monomslist,varseval)
        #show(q)
        fOAs=fOAs+[q]
        fODs=fODs+[optimaldistsq]
    #
    fOGs=[f,fOAs[0]]
    for whichOA in range(1,len(fOAs)):
        fOGs=fOGs+[fOAs[whichOA]-fOAs[whichOA-1]]
    #
    alltheOGs=alltheOGs+[fOGs]
        #DSOGs=GS_UpTo(monomslist,g,15,monomslist,varseval)
    deg=0
    print("\\begin{align*}")
    print('p_{',deg,'}&=',latex(fOAs[0]),'\\\\')
    for deg in range(1,len(fOAs)):
        if fOAs[deg]!=fOAs[deg-1]:
            print('p_{',deg,'}&=',latex(fOAs[deg]),'\\\\')
            print('%%%%%%%%%%%%%%%%%%%')
        #deg=deg+1
    print("\\end{align*}")
    print("")
    deg=0
    print("\\begin{align*}")
    print('\\nu_{',deg,'}&=',latex(fODs[0]),'\\\\')
    for deg in range(1,len(fODs)):
        if fODs[deg]!=fODs[deg-1]:
            print('\\nu_{',deg,'}&=',latex(fODs[deg]),'\\\\')
            print('%%%%%%%%%%%%%%%%%%%')
        #deg=deg+1
    print("\\end{align*}")
    print("")
    print("")
    print("")

In [ ]:
orthogs=[fOAs[0]]
for index in range(1,len(fOAs)):
    newOG=fOAs[index]-fOAs[index-1]
    #print(newOG)
    orthogs=orthogs+[newOG]
deg=0
print("\\begin{align*}")
print('\\phi_{',deg,'}&=',latex(orthogs[0]),'\\\\')
for deg in range(1,len(orthogs)):
    #if orthogs[deg]!=orthogs[deg-1]:
        print('\\phi_{',deg,'}&=',latex(orthogs[deg]),'\\\\')
        print('%%%%%%%%%%%%%%%%%%%')
    #deg=deg+1
print("\\end{align*}")

In [ ]:
for orthogs in alltheOGs:
    show(orthogs[0])
    print("test function")
    print("\\begin{equation*}")
    print("\scalebox{2}{\ensuremath{f=",latex(orthogs[0]),"}}")
    print("\\end{equation*}")
    #
    print("\\begin{align*}")
    for deg in range(1,len(orthogs)):
        print('\\phi_{',deg-1,'}&=',latex(orthogs[deg]),'\\\\')
        print('%%%%%%%%%%%%%%%%%%%')
    print("\\end{align*}")

In [ ]:
show(154572*fOAs[15])

In [ ]:
for g in fOAs:
    thingtonorm=g*f-1
    distnorm=InnerProd(thingtonorm,thingtonorm,monomslist,varseval)
    show(distnorm)

note to self: attempt nikolski examples with generic lambda